# The Atlas API

In this tutorial we'll learn the basic functionalities of `Atlas` that can be used to define, manipulate and run `generators`

--------------------------
## Chapter 1 : Bare Basics

### 1.1 Defining a Generator

Let us define a generator for building binary strings (containing 0s and 1s) of a certain length. We write a simple function that uses a loop to `Select` either `"0"` or `"1"` for each bit separately.

In [1]:
from atlas import generator

@generator
def binary(length: int):
    s = ""
    for _ in range(length):
        s += Select(["0", "1"])
        
    return s

### 1.2 Enumerating a Generator

Let us use this generator to enumerate all binary strings of length `2`

In [2]:
for s in binary.generate(2):
    print(s)

00
01
10
11


-------------------------
## Chapter 2 : Introduction to Strategies and Operators

The core building block behind generators is a `Strategy`. A strategy essentially defines the behavior of the generator. The `binary` generator we defined above used a `depth-first enumeration` strategy that explored all possible executions of the generator.

A strategy itself is simply a collection of behaviors for the `operators` used inside the generator. In the `binary` example above, the depth-first strategy defines the semantics of the `Select` operator that allows it to systematically explore both the choices (`"0"` and `"1"`).

Atlas provides an API that allows you to change the strategy used by the generator, as well as define your own strategies. Let us try changing the behavior of the `binary` generator to randomly return binary strings of the given length.

### 2.1 Changing the Strategy used by a Generator

#### Method 1 : Redefining the Generator

In [3]:
@generator(strategy='randomized')
def binary(length: int):
    s = ""
    for _ in range(length):
        s += Select(["0", "1"])
        
    return s

for s in binary.generate(2):
    print(s)
    break

01


Notice that we are breaking the loop after one iteration. This is because when using randomized strategies, a generator is an infinite iterator. To avoid using loop constructs when using such randomized generators, `Atlas` also provides a convenience method `call` that simply returns a value based on a random execution of the generator.

In [4]:
print(binary.call(2))
print(binary.call(2))
print(binary.call(2))
print(binary.call(2))

01
11
01
00


#### Method 2 : Using `set_default_strategy`

Redefining the generator is obviously not the way to go when dealing with large generators. `Atlas` provides the `set_default_strategy` method that can be used to set the strategy for a generator without redefining it.

In [5]:
print("------------")
print("DFS Strategy")
print("------------")

binary.set_default_strategy('dfs')
for s in binary.generate(2):
    print(s)
   
print("-------------------")
print("Randomized Strategy")
print("-------------------")

binary.set_default_strategy('randomized')

print(binary.call(2))
print(binary.call(2))
print(binary.call(2))
print(binary.call(2))

------------
DFS Strategy
------------
00
01
10
11
-------------------
Randomized Strategy
-------------------
10
00
11
00


#### Method 3 : Using `with_env`

Strategies can also be set *temporarily* without changing the default strategy by calling the `with_env` method along with the `strategy` argument, after calling `generate` as in section `1.2`.

In [6]:
binary.set_default_strategy('randomized')
print(f"Randomized - {binary.call(2)}")

for s in binary.with_env(strategy='dfs').generate(2):
    print(f"DFS - {s}")
    
print(f"Randomized - {binary.call(2)}")

Randomized - 10
DFS - 00
DFS - 01
DFS - 10
DFS - 11
Randomized - 11


## 2.2 Defining your Own Strategies / Operators

`Atlas` aims to be a framework offering complete flexibility and control over generators. Clearly, in order to achieve this, control over strategies is necessary. `Atlas` makes it easy to create custom strategies (and consequently operators) to use in your generator. As an example, we'll a create a strategy that does DFS in *reverse* i.e. reverses the order of exploration. We will also introduce a new operator `SelectReversed`.

In [8]:
from atlas.operators import operator
from atlas.strategies import DfsStrategy

class ReversedDfs(DfsStrategy):
    @operator
    def SelectReversed(self, domain, **kwargs):
        yield from reversed(domain)
        
        
@generator
def binary(length: int):
    s = ""
    for _ in range(length):
        s += SelectReversed(["0", "1"])
        
    return s
        
    
for s in binary.with_env(strategy=ReversedDfs()).generate(2):
    print(f"Reversed DFS - {s}")

Reversed DFS - 11
Reversed DFS - 10
Reversed DFS - 01
Reversed DFS - 00


Operator definitions for strategies that implement `DfsStrategy` need to return an iterator/iterable. We use the native Python generator syntax (using `yield`) that helps us conveniently define this iterator.

Let us define a new randomized strategy that uses biased coin tosses to generate the binary strings

In [9]:
from atlas.strategies import RandStrategy
import numpy as np

class CoinTossingStrategy(RandStrategy):
    @operator
    def CoinToss(self, bias, **kwargs):
        return np.random.choice(["0", "1"], p=[1-bias, bias])
        
        
@generator
def binary(length: int):
    s = ""
    for _ in range(length):
        s += CoinToss(bias=0.75)
        
    return s

binary.set_default_strategy(CoinTossingStrategy())
print(binary.call(5))
print(binary.call(5))

11101
11111


Note how randomized strategies don't need to define iterators. Strategies sub-classing `RandStrategy` can define operators as regular functions. Also note we pass `bias` as a keyword argument in contrast to the `domain` passed as a positional argument in the previous example. In general, operators can use any number of keyword arguments and as long as the generator calls them correctly, `Atlas` will correctly pass the arguments to the operator. However the first positional argument will be stored in the `domain` argument. Also, operators **need** to have a `**kwargs` parameter in order to function correctly.